In [2]:
#Importamos las librerias
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import time
from numpy import linalg as LA
import random
from time import time
%matplotlib qt

In [3]:
#Definimos la funcion a minimizar
def f(x):
    fx = 100*(np.sqrt(x[0]**2+(x[1]+1)**2)-1)**2 + 90*(np.sqrt(x[0]**2+(x[1]+1)**2)-1)**2 -(20*x[0]+40*x[1])
    return fx

In [4]:
#Definir funcion para calcular la matriz del gradiente
def gradient(x,delta):
    grad=np.zeros(2)
    grad[0]=(f([x[0]+delta,x[1]])- f([x[0]-delta,x[1]]))/(2*delta)
    grad[1]=(f([x[0],x[1]+delta])- f([x[0],x[1]-delta]))/(2*delta)
    return grad

In [5]:
#Definir funcion para calcular la matriz Hessiana
def Hessian(x,delta):
    H=np.zeros([2,2])
    H[0,0]= (f([x[0]+delta,x[1]]) - 2*f([x[0],x[1]]) +  f([x[0]-delta,x[1]]))/(delta**2)
    H[1,1]= (f([x[0],x[1]+delta]) - 2*f([x[0],x[1]]) +  f([x[0],x[1]-delta]))/(delta**2)
    H[0,1]= (f([x[0]+delta,x[1]+delta]) - f([x[0]+delta,x[1]-delta]) - f([x[0]-delta,x[1]+delta]) +  f([x[0]-delta,x[1]-delta]))/(4*delta**2)
    H[1,0]= H[0,1]
    return H

In [6]:
#Utilizamos el algoritmo de golden section para encontrar alpha optimo
def golden(x,search,xi,eps):
    a = xi[0];
    b = xi[1];
    tau = 0.381967;
    alpha1 = a*(1-tau) + b*tau;
    alpha2 = a*tau + b*(1-tau);
    falpha1 = f(x+alpha1*search);
    falpha2 = f(x+alpha2*search);
    for i in range(100):
        if falpha1 > falpha2:
            a = alpha1;
            alpha1 = alpha2;
            falpha1 = falpha2;
            alpha2 = tau*a + (1-tau)*b;
            falpha2 = f(x+alpha2*search);
        else:
            b = alpha2;
            alpha2 = alpha1;
            falpha2 = falpha1;
            alpha1 = tau*b + (1-tau)*a;
            falpha1 = f(x+alpha1*search);
        if np.abs(f(x+alpha1*search)- f(x+alpha2*search)) < eps :
            break;
    return alpha1,falpha1

In [7]:
#Graficamos la funcion de prueba
X1 = np.arange(-5, 5, 0.01)
X2 = np.arange(-5, 5, 0.01)
x1, x2 = np.meshgrid(X1, X2)
z = f([x1,x2])
fig = plt.figure()
ax = fig.add_subplot(projection = '3d')
surf = ax.plot_surface(x1, x2, z, cmap=cm.autumn, linewidth=0, antialiased = False, alpha=0.2)
plt.show()

In [8]:
#Variable en comun de los métodos
delta = 1e-3 
ep1 = 1e-3
puntosSD = []

In [9]:
def graficaUnaaUna(puntos, nombreMetodo):
    #Generamos valores ramdom para generar un color diferente de grafica
    r = random.random()
    b = random.random()
    g = random.random()
    color = (r, g, b)
    aux = 0
    x = np.linspace(-1, 1, 100)
    y = np.linspace(-1, 1, 100)
    xx, yy = np.meshgrid(x, y)
    z = f([xx, yy])
    plt.contour(x,y,z,20)
    plt.plot(-1,-1, 'ro--',  markersize=6)
    plt.plot(0.5,0, 'go--', markersize=6)
    #Graficamos los trazos (direcciones) que va generando el algoritmo
    for pts in puntos:
        if(aux==len(puntos)-1):
            plt.plot(pts[0],pts[1], c=color, linewidth=2, label=str(nombreMetodo))
        else:
            plt.plot(pts[0],pts[1], c=color, linewidth=2)
        aux = aux + 1
    plt.legend()    
    plt.show

In [10]:
start_time = time() #Conteo para tiempo
iteraciones = 0 #Conteo de numero de iteraciones
xi_sd = [-1,1] #Punto de partida
xi_1 = [-1,1] #Punto inicial para el metodo del golden section
fx_value = f(xi_sd) #Se evalua la funcion con el punto inicial
print('Valor de la función inicial = %f ' % fx_value)
for i in range(1000): #Proceso iterativo del algoritmo
    grad = gradient(xi_1,delta) #Se obtiene el vector del gradiente de la funcion objetivo
    si = -grad #La direccion de busqueda es el gradiente negativo
    alpha, fx_curr = golden(xi_1,si,xi_sd,ep1) #Se encuentra el alpha optimo (tamaño de paso)
    x_opt = xi_1 + alpha*si #Se encuentra el nuevo optimo
    if abs(fx_curr-fx_value)<ep1 or LA.norm(grad)<ep1: #Establecemos como condicion de paro, que la diferencia fx_anterior-fx_actual sea menor a epsilon o que la norma del gradiente sea menor a epsilon
        print('{0}\t[{1:.4f},{2:.4f}]\t\t{3:.4f}\t\t{4:.4f}'.format(i, x_opt[0], x_opt[1],fx_value,LA.norm(grad)))
        break
    fx_value=fx_curr #Ahora fx_value se iguala con el actual, de esta forma el actual ahora es el anterior
    print('{0}\t[{1:.4f},{2:.4f}]\t\t{3:.4f}\t\t{4:.4f}'.format(i, x_opt[0], x_opt[1],fx_value,LA.norm(grad)))
    puntosSD.append([[xi_1[0],x_opt[0]],[xi_1[1],x_opt[1]]]) #Almacenamos los punto de direccion del algoritmo
    xi_1=x_opt #ahora x anterior es x_opt
    iteraciones += 1
elapsed_time = (time() - start_time)*1000
print("Tiempo transcurrido: %f" % elapsed_time)
print("Iteraciones: %d" % iteraciones)
#Graficamos el resultado del Steepest descent method
graficaUnaaUna(puntosSD, 'Steepest descent method')

Valor de la función inicial = 270.294169 
0	[-0.3892,-0.0092]		8.9425		444.3158
1	[-0.0345,0.2013]		0.3751		33.6772
2	[0.0410,0.0766]		-2.7472		42.8521
3	[0.1616,0.1444]		-4.3989		21.6597
4	[0.2127,0.0633]		-5.4337		21.9813
5	[0.2783,0.1043]		-6.0764		16.1606
6	[0.3148,0.0471]		-6.5224		13.2170
7	[0.3540,0.0712]		-6.8064		11.4880
8	[0.3868,0.0277]		-7.0162		7.8351
9	[0.4105,0.0454]		-7.1464		8.5604
10	[0.4349,0.0157]		-7.2356		4.5741
11	[0.4483,0.0261]		-7.2851		5.4709
12	[0.4691,0.0061]		-7.3234		2.4656
13	[0.4763,0.0126]		-7.3382		3.1097
14	[0.4833,0.0040]		-7.3451		1.1885
15	[0.4869,0.0067]		-7.3485		1.4562
16	[0.4926,0.0006]		-7.3509		0.6321
17	[0.4930,0.0011]		-7.3509		0.9605
Tiempo transcurrido: 10.963678
Iteraciones: 17
